In [23]:
import os
import sys
import pandas as pd
import logging
import json
import uuid
import db_dtypes
from datetime import date, datetime
from google.cloud import bigquery
from hashlib import md5
from typing import List


# SETUP

DATA_DIR = "data/air_travel/"
DEFAULT_TICKETS_FILE = os.path.join(DATA_DIR, "tickets.json") 
PROJECT_NAME = "deb-01-372120"
DATASET_NAME = "air_travel"


# **** TABLE SCHEMAS ****

TABLE_METADATA = {
    'airlines': {
        'table_name': 'airlines',
        'schema': [
            bigquery.SchemaField('name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('iata', 'string', mode='REQUIRED'),
            bigquery.SchemaField('icao', 'string', mode='REQUIRED'),
            bigquery.SchemaField('callsign', 'string', mode='REQUIRED'),
            bigquery.SchemaField('country', 'string', mode='REQUIRED')
        ]
    },
    'airports': {
        'table_name': 'airports',
        'schema': [
            bigquery.SchemaField('name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('city', 'string', mode='REQUIRED'),
            bigquery.SchemaField('country', 'string', mode='REQUIRED'),
            bigquery.SchemaField('origin_destination', 'string', mode='REQUIRED'),
            bigquery.SchemaField('icao', 'string', mode='REQUIRED'),
            bigquery.SchemaField('latitude', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('longitude', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('altitude', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('tz_timezone', 'string', mode='REQUIRED')
        ]
    },
    'passengers': {
        'table_name': 'passengers',
        'schema': [
            bigquery.SchemaField('first_name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('last_name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('gender', 'string', mode='REQUIRED'),
            bigquery.SchemaField('birth_date', 'datetime', mode='REQUIRED'),
            bigquery.SchemaField('email', 'string', mode='REQUIRED'),
            bigquery.SchemaField('street','string', mode='REQUIRED'),
            bigquery.SchemaField('city', 'string', mode='REQUIRED'),
            bigquery.SchemaField('state', 'string', mode='REQUIRED'),
            bigquery.SchemaField('zip', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('start_date', 'datetime', mode='NULLABLE'),
            bigquery.SchemaField('end_date', 'datetime', mode='NULLABLE'),
            bigquery.SchemaField('passengers_sk', 'string', mode='REQUIRED')
        ]
    },
    'tickets': {
        'table_name': 'tickets',
        'schema': [
            bigquery.SchemaField('eticket_num', 'string', mode='REQUIRED'),
            bigquery.SchemaField('confirmation', 'string', mode='REQUIRED'),
            bigquery.SchemaField('ticket_date', 'datetime', mode='REQUIRED'),
            bigquery.SchemaField('price', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('seat', 'string', mode='REQUIRED'),
            bigquery.SchemaField('status','string', mode='REQUIRED'),
            bigquery.SchemaField('airline', 'string', mode='REQUIRED'), 
            bigquery.SchemaField('origin', 'string', mode='REQUIRED'),
            bigquery.SchemaField('destination', 'string', mode='REQUIRED'),
            bigquery.SchemaField('passengers_sk', 'string', mode='REQUIRED')
        ]
    }
}


# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")
client = bigquery.Client()

logger.info(f"Setup Completed")


[DEBUG][2023-01-31 22:37:51,898][845468911:0096] : Creating bigquery client
[INFO ][2023-01-31 22:37:51,907][845468911:0099] : Setup Completed


In [4]:
# create dataset 
dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)

logger.info(f"Created air travel dataset: {dataset.full_dataset_id}")

[INFO ][2023-01-31 22:26:19,313][2632008059:0007] : Created air travel dataset: deb-01-372120:air_travel


In [5]:
# air travel data file name
filename = DEFAULT_TICKETS_FILE
logger.debug(f"attempting to process: {filename}")

# check if the file exists
assert os.path.exists(filename), f"Data file does not exists: '{filename}'"
# check if the file contains any data 
assert os.path.getsize(filename) > 78, f"Data file size incorrect; does not seem to contain data: '{filename}'"

# convert json into an array and then load into dataframe
data = [json.loads(line) for line in open(filename, 'r')]

df = pd.json_normalize(data)
logger.info(f"loaded {len(df.index)} rows from: {filename}")

# check schema: contains all expected columns?
expected_columns = [
    'eticket_num',
    'confirmation',
    'ticket_date',
    'price',
    'seat',
    'status',
    'airline.name',
    'airline.iata',
    'airline.icao',
    'airline.callsign',
    'airline.country',
    'origin.name',
    'origin.city',
    'origin.country', 
    'origin.iata', 
    'origin.icao', 
    'origin.latitude', 
    'origin.longitude', 
    'origin.altitude', 
    'origin.tz_timezone', 
    'destination.name', 
    'destination.city', 
    'destination.country', 
    'destination.iata', 
    'destination.icao', 
    'destination.latitude', 
    'destination.longitude', 
    'destination.altitude', 
    'destination.tz_timezone', 
    'passenger.first_name', 
    'passenger.last_name', 
    'passenger.gender', 
    'passenger.birth_date', 
    'passenger.email', 
    'passenger.street', 
    'passenger.city', 
    'passenger.state', 
    'passenger.zip'
    ]
for col in expected_columns:
    assert col in list(df.columns), f"Data file missing required column: {col}"

# assign & remember receipts dataframe
tickets_df = df
display(tickets_df.head(n=10))

[DEBUG][2023-01-31 22:26:19,411][784627330:0003] : attempting to process: data/air_travel/tickets.json
[INFO ][2023-01-31 22:26:19,891][784627330:0014] : loaded 4096 rows from: data/air_travel/tickets.json


,eticket_num,confirmation,ticket_date,price,seat,status,airline.name,airline.iata,airline.icao,airline.callsign,...,passenger.last_name,passenger.gender,passenger.birth_date,passenger.email,passenger.street,passenger.city,passenger.state,passenger.zip,origin,destination
0,498-938211-0795,ZVFDC4,2022-03-23,723.42,31I,active,China Eastern Airlines,MU,CES,CHINA EASTERN,...,Brown,M,1969-02-17,robert.brown.69@hotmail.com,5007 Thomas Way,Lake Hollystad,DC,20027,NaN,NaN
1,482-850738-6048,IL5GUI,2022-03-23,765.18,29B,active,Hawaiian Airlines,HA,HAL,HAWAIIAN,...,Kent,F,1998-08-05,laura.kent.98@hotmail.com,13991 Davis Village,North Catherineborough,PA,16516,NaN,NaN
2,275-207321-8092,CYEFBC,2022-03-21,753.89,26I,active,Wizz Air,W6,WZZ,WIZZ AIR,...,Tucker,F,1965-01-22,lisa.tucker.65@hotmail.com,04135 Marvin Via,North Kristabury,MA,01093,NaN,NaN
3,246-793315-3102,ZNGPC2,2022-03-22,793.89,15A,active,AirAsia,AK,AXM,ASIAN EXPRESS,...,Yates,NB,1975-03-31,matthew.yates.75@yahoo.com,76045 Samantha Road Suite 111,Lake Jeffrey,DE,19898,NaN,NaN
4,091-128904-1226,MGSBD9,2022-03-24,820.25,17F,active,Xiamen Airlines,MF,CXA,XIAMEN AIR,...,Villanueva,NB,1945-08-14,megan.villanueva.45@hotmail.com,848 Melissa Springs Suite 947,Kellerstad,TX,76177,NaN,NaN
5,115-196069-8963,XFYQC0,2022-03-23,892.69,18C,active,Air New Zealand,NZ,ANZ,NEW ZEALAND,...,Hall,NB,1944-08-31,sarah.hall.44@gmail.com,75420 Michael Mountains Suite 485,New Victoria,HI,96727,NaN,NaN
6,396-673460-1326,N5UOOZ,2022-03-23,889.53,3C,active,Jeju Air,7C,JJA,JEJU AIR,...,Thompson,M,1968-05-02,seth.thompson.68@yahoo.com,22455 Higgins Junction Apt. 042,New Keith,OR,97405,NaN,NaN
7,380-894599-8109,PAA19Y,2022-03-22,706.78,7D,active,American Airlines,AA,AAL,AMERICAN,...,Garcia,F,1950-02-12,jennifer.garcia.50@gmail.com,6607 Sharp Common,Chadstad,VA,22121,NaN,NaN
8,614-960971-2686,EF4BHJ,2022-03-23,486.40,24J,active,Juneyao Airlines,HO,DKH,JUNEYAO AIRLINES,...,Clark,F,1991-11-09,becky.clark.91@gmail.com,691 Jones Cliffs,Michaelburgh,TX,76003,NaN,NaN
9,481-321233-0702,FVM9EE,2022-03-23,855.93,16A,active,Royal Air Maroc,AT,RAM,ROYALAIR MAROC,...,Cook,M,1976-07-29,ronald.cook.76@hotmail.com,93328 Davis Island,Rodriguezside,MD,21408,NaN,NaN


In [6]:
# Gather unique airlines 
# start from the tickets_df
df = tickets_df

logger.debug(f"getting unique airlines...")

# set of unique columns to return
cols = [
  'airline.name', 
  'airline.iata', 
  'airline.icao', 
  'airline.callsign', 
  'airline.country'
  ]
# group by unique columns and only select them
df = df.groupby(cols).all()
df = df.reset_index().loc[:, cols]
# rename columns
df = df.rename(columns={
  'airline.name': 'name', 
  'airline.iata':'iata', 
  'airline.icao':'icao', 
  'airline.callsign':'callsign', 
  'airline.country':'country'
  })

# Set index to IATA
df = df.set_index(keys='iata')

logger.info(f"airlines dim - found {len(df.index)} rows")
df.head(10)

[DEBUG][2023-01-31 22:26:20,057][2459079849:0005] : getting unique airlines...
[INFO ][2023-01-31 22:26:20,205][2459079849:0030] : airlines dim - found 48 rows


,name,icao,callsign,country
iata,,,,
AC,Air Canada,ACA,AIR CANADA,Canada
CA,Air China,CCA,AIR CHINA,China
AF,Air France,AFR,AIRFRANS,France
NZ,Air New Zealand,ANZ,NEW ZEALAND,New Zealand
AK,AirAsia,AXM,ASIAN EXPRESS,Malaysia
AS,Alaska Airlines,ASA,Inc.,ALASKA
G4,Allegiant Air,AAY,ALLEGIANT,United States
AA,American Airlines,AAL,AMERICAN,United States
BA,British Airways,BAW,SPEEDBIRD,United Kingdom


In [7]:
# Create load_table function that will deal with loading airline dataframe to BigQuery

def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

In [8]:
# Load airline table to BigQuery
# get table name and schema from our TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['airlines']['table_name']}"
schema = schema=TABLE_METADATA['airlines']['schema']
# load dataframe
load_table(df, client, table_name, schema)

logger.info(f"loaded airlines dim")

[INFO ][2023-01-31 22:26:20,308][315478070:0031] : loading table: 'deb-01-372120.air_travel.airlines'
[INFO ][2023-01-31 22:26:24,375][315478070:0036] : loaded 48 rows into deb-01-372120:air_travel.airlines
[INFO ][2023-01-31 22:26:24,376][1629074712:0008] : loaded airlines dim


In [9]:
# Gather unique aiports 
# start from the tickets_df
df = tickets_df

# Concatenate origin and desination iata codes
df['origin_destination'] = df['origin.iata'] + '-' + df['destination.iata']

logger.debug(f"getting unique airports...")

# Set of unique columns to return
cols = [
  'origin.name', 
  'origin.city', 
  'origin.country',
  'origin_destination',  
  'origin.icao', 
  'origin.latitude', 
  'origin.longitude', 
  'origin.altitude', 
  'origin.tz_timezone'
  ]
# group by unique columns and only select them
df = df.groupby(cols).all()
df = df.reset_index().loc[:, cols]
# rename columns
df = df.rename(columns={
  'origin.name':'name',
  'origin.city':'city', 
  'origin.country':'country', 
  'origin.icao':'icao', 
  'origin.latitude':'latitude', 
  'origin.longitude':'longitude', 
  'origin.altitude':'altitude', 
  'origin.tz_timezone':'tz_timezone'
  })

# Set index to IATA
df = df.set_index(keys='origin_destination')

logger.info(f"airports dim - found {len(df.index)} rows")
df.head(10)

[DEBUG][2023-01-31 22:26:24,491][2285433002:0008] : getting unique airports...
[INFO ][2023-01-31 22:26:24,707][2285433002:0040] : airports dim - found 3876 rows


,name,city,country,icao,latitude,longitude,altitude,tz_timezone
origin_destination,,,,,,,,
AUH-CAN,Abu Dhabi International Airport,Abu Dhabi,United Arab Emirates,OMAA,24.43,54.65,88.0,Asia/Dubai
AUH-CBB,Abu Dhabi International Airport,Abu Dhabi,United Arab Emirates,OMAA,24.43,54.65,88.0,Asia/Dubai
AUH-JED,Abu Dhabi International Airport,Abu Dhabi,United Arab Emirates,OMAA,24.43,54.65,88.0,Asia/Dubai
AUH-JJN,Abu Dhabi International Airport,Abu Dhabi,United Arab Emirates,OMAA,24.43,54.65,88.0,Asia/Dubai
AUH-MDE,Abu Dhabi International Airport,Abu Dhabi,United Arab Emirates,OMAA,24.43,54.65,88.0,Asia/Dubai
AUH-VDC,Abu Dhabi International Airport,Abu Dhabi,United Arab Emirates,OMAA,24.43,54.65,88.0,Asia/Dubai
MAD-BHX,Adolfo Suárez Madrid–Barajas Airport,Madrid,Spain,LEMD,40.47,-3.56,1998.0,Europe/Madrid
MAD-CDG,Adolfo Suárez Madrid–Barajas Airport,Madrid,Spain,LEMD,40.47,-3.56,1998.0,Europe/Madrid
MAD-CKG,Adolfo Suárez Madrid–Barajas Airport,Madrid,Spain,LEMD,40.47,-3.56,1998.0,Europe/Madrid


In [10]:
# Load airport dim into BigQuery
# get table name and schema from our TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['airports']['table_name']}"
schema = schema=TABLE_METADATA['airports']['schema']
# load dataframe
load_table(df, client, table_name, schema)

logger.info(f"loaded airports dim")

[INFO ][2023-01-31 22:26:24,786][315478070:0031] : loading table: 'deb-01-372120.air_travel.airports'
[INFO ][2023-01-31 22:26:30,350][315478070:0036] : loaded 3876 rows into deb-01-372120:air_travel.airports
[INFO ][2023-01-31 22:26:30,351][3846598676:0008] : loaded airports dim


In [11]:
# Gather unique passengers 
# start from the tickets_df
df = tickets_df

logger.debug(f"getting unique passengers...")

# set of unique columns to return
cols = [
  'passenger.first_name', 
  'passenger.last_name',
  'passenger.gender', 
  'passenger.birth_date', 
  'passenger.email',
  'passenger.street',
  'passenger.city',
  'passenger.state',
  'passenger.zip'
  ]
# group by unique columns and only select them
df = df.groupby(cols).all()
df = df.reset_index().loc[:, cols]
# rename columns
df = df.rename(columns={
  'passenger.first_name':'first_name',
  'passenger.last_name':'last_name',
  'passenger.gender':'gender',
  'passenger.birth_date':'birth_date',
  'passenger.email':'email',
  'passenger.street':'street',
  'passenger.city':'city',
  'passenger.state':'state',
  'passenger.zip':'zip'
  })



# Add start and end date columns
df['start_date'] = date.today()
df['end_date'] = None

# Generate UUID for passengers df
# Create UUID Series
df['passengers_sk'] = pd.Series([uuid.uuid4() for i in range(len(df.index))])

logger.info(f"generated passengers uuids")

# Set index to email
df = df.set_index(keys='email')

logger.info(f"passengers dim - found {len(df.index)} rows")
df.head(5)

[DEBUG][2023-01-31 22:26:30,388][2831550189:0005] : getting unique passengers...
[INFO ][2023-01-31 22:26:30,473][2831550189:0045] : generated passengers uuids
[INFO ][2023-01-31 22:26:30,475][2831550189:0050] : passengers dim - found 32 rows


,first_name,last_name,gender,birth_date,street,city,state,zip,start_date,end_date,passengers_sk
email,,,,,,,,,,,
annette.hawkins.43@yahoo.com,Annette,Hawkins,F,1943-07-11,361 Robinson Green Apt. 635,North Lynntown,NV,89825,2023-01-31,None,d77173d7-ee83-4b8d-b09b-cdddbdad8713
autumn.morse.60@hotmail.com,Autumn,Morse,F,1960-01-18,6984 Price Shoals,Erictown,HI,96818,2023-01-31,None,d5b9f3ab-25dc-4f81-9ae1-5a69dbc5d877
becky.clark.91@gmail.com,Becky,Clark,F,1991-11-09,691 Jones Cliffs,Michaelburgh,TX,76003,2023-01-31,None,05dfbb6d-cfcb-4d4a-a882-f337995c779c
belinda.cook.91@hotmail.com,Belinda,Cook,F,1991-01-26,1965 Kelly Field Apt. 094,Jonesberg,IL,60613,2023-01-31,None,65331a94-75eb-4d7b-b50d-8cff123f4d28
carl.wilson.80@hotmail.com,Carl,Wilson,M,1980-04-24,2814 Houston Hills,Rodriguezside,IA,51971,2023-01-31,None,f5f10a0d-702a-4f2c-bc05-549cf59de6c4


In [12]:
# Check the data types of the schema
print(df.dtypes)

first_name       object
last_name        object
gender           object
birth_date       object
street           object
city             object
state            object
zip              object
start_date       object
end_date         object
passengers_sk    object
dtype: object


In [13]:
# Force the data types of the dated columns to match the schema requirements
df[['birth_date', 'start_date', 'end_date']] = df[['birth_date', 'start_date', 'end_date']].astype('datetime64[ns]')

# Force the data types of the string category to match their schema requirements
df[['first_name', 'last_name', 'gender', 'street', 'city', 'state', 'passengers_sk']] = df[['first_name', 'last_name', 'gender', 'street', 'city', 'state', 'passengers_sk']].astype('string')

# Force data type of zip to be int64
df['zip'] = df['zip'].astype('int64')

# Print datatypes of columns
(df.dtypes)


first_name               string
last_name                string
gender                   string
birth_date       datetime64[ns]
street                   string
city                     string
state                    string
zip                       int64
start_date       datetime64[ns]
end_date         datetime64[ns]
passengers_sk            string
dtype: object

In [14]:
# Load passengers dim into BigQuery
# get table name and schema from our TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['passengers']['table_name']}"
schema = schema=TABLE_METADATA['passengers']['schema']
# load dataframe
load_table(df, client, table_name, schema)

logger.info(f"loaded passengers dim")

[INFO ][2023-01-31 22:26:30,647][315478070:0031] : loading table: 'deb-01-372120.air_travel.passengers'
[INFO ][2023-01-31 22:26:35,454][315478070:0036] : loaded 32 rows into deb-01-372120:air_travel.passengers
[INFO ][2023-01-31 22:26:35,456][251488372:0008] : loaded passengers dim


In [15]:
# Set Tickets Fact table with all columns except passenger
df = tickets_df
df = df[['eticket_num', 'confirmation', 'price', 'ticket_date', 'seat', 'status', 'airline.iata', 'origin.iata', 'destination.iata']]

# Set index
df = df.set_index(keys='eticket_num')

# Rename columns
df = df.rename(columns={'airline.iata':'airline', 'origin.iata':'origin', 'destination.iata':'destination'})

logger.info(f"tickets fact - found {len(df.index)} rows")
df.head(10)

[INFO ][2023-01-31 22:26:35,514][3031790187:0011] : tickets fact - found 4096 rows


,eticket_num,confirmation,price,ticket_date,seat,status,airline,origin,destination
0,498-938211-0795,ZVFDC4,723.42,2022-03-23,31I,active,MU,YUL,MDW
1,482-850738-6048,IL5GUI,765.18,2022-03-23,29B,active,HA,KWE,MNL
2,275-207321-8092,CYEFBC,753.89,2022-03-21,26I,active,W6,PVR,IBZ
3,246-793315-3102,ZNGPC2,793.89,2022-03-22,15A,active,AK,PMC,GDN
4,091-128904-1226,MGSBD9,820.25,2022-03-24,17F,active,MF,BWI,LGW
5,115-196069-8963,XFYQC0,892.69,2022-03-23,18C,active,NZ,ROV,CVG
6,396-673460-1326,N5UOOZ,889.53,2022-03-23,3C,active,7C,YWG,OKA
7,380-894599-8109,PAA19Y,706.78,2022-03-22,7D,active,AA,ONT,RGN
8,614-960971-2686,EF4BHJ,486.40,2022-03-23,24J,active,HO,SJU,SVX
9,481-321233-0702,FVM9EE,855.93,2022-03-23,16A,active,AT,YEG,TYN


In [18]:
# Lookup passenger UUID and join it to the tickets df
query = f"""
  SELECT
    passengers_sk
  FROM
    `{PROJECT_NAME}.{DATASET_NAME}.passengers`
"""
# 
pass_df = client.query(query).to_dataframe()

df = df.drop(columns='passengers_sk', errors='ignore')

df = df.join(pass_df, how='inner')

# Set index
df = df.set_index(keys='eticket_num')

df.head()

,confirmation,price,ticket_date,seat,status,airline,origin,destination,passengers_sk
eticket_num,,,,,,,,,
498-938211-0795,ZVFDC4,723.42,2022-03-23,31I,active,MU,YUL,MDW,d77173d7-ee83-4b8d-b09b-cdddbdad8713
482-850738-6048,IL5GUI,765.18,2022-03-23,29B,active,HA,KWE,MNL,d5b9f3ab-25dc-4f81-9ae1-5a69dbc5d877
275-207321-8092,CYEFBC,753.89,2022-03-21,26I,active,W6,PVR,IBZ,05dfbb6d-cfcb-4d4a-a882-f337995c779c
246-793315-3102,ZNGPC2,793.89,2022-03-22,15A,active,AK,PMC,GDN,65331a94-75eb-4d7b-b50d-8cff123f4d28
091-128904-1226,MGSBD9,820.25,2022-03-24,17F,active,MF,BWI,LGW,780ae4b4-c510-4297-8ffb-2d4a266dce49


In [19]:
df.dtypes

confirmation      object
price            float64
ticket_date       object
seat              object
status            object
airline           object
origin            object
destination       object
passengers_sk     object
dtype: object

In [21]:
# Explicity set data types
df[['confirmation', 'seat', 'status', 'airline', 'origin', 'destination', 'passengers_sk']] = df[['confirmation', 'seat', 'status', 'airline', 'origin', 'destination', 'passengers_sk']].astype('string')

df['ticket_date'] = df['ticket_date'].astype('datetime64[ns]')

print(df.dtypes)

confirmation             string
price                   float64
ticket_date      datetime64[ns]
seat                     string
status                   string
airline                  string
origin                   string
destination              string
passengers_sk            string
dtype: object


In [24]:
# Load tickets fact table to BigQuery
# get table name and schema from our TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['tickets']['table_name']}"
schema = schema=TABLE_METADATA['tickets']['schema']
# load dataframe
load_table(df, client, table_name, schema)

logger.info(f"loaded tickets dim")

[INFO ][2023-01-31 22:37:56,210][315478070:0031] : loading table: 'deb-01-372120.air_travel.tickets'
[INFO ][2023-01-31 22:38:00,925][315478070:0036] : loaded 32 rows into deb-01-372120:air_travel.tickets
[INFO ][2023-01-31 22:38:00,927][1578788439:0008] : loaded tickets dim
